In [1]:
!export LC_ALL=en_US.UTF-8
import tensorflow as tf
import json
from pprint import pprint
import re
import numpy as np
from embedding import *
import nltk

In [2]:
data = json.load(open('../train-v1.1.json'))

In [3]:
from nltk.corpus import stopwords
#nltk.download('stopwords')
nltkStopWords = stopwords.words('english')

In [4]:
def extractor(data):
    contexts = []
    qas = []
    for i in range(len(data["data"])):
        for j in range(len(data["data"][i]["paragraphs"])):
            contexts.append(data["data"][i]["paragraphs"][j]["context"])
            qas.append(data["data"][i]["paragraphs"][j]["qas"])
    return (contexts,qas)

In [5]:
CapPassage = False

from nltk.tokenize import word_tokenize
contexts,qas = extractor(data)

def find_sub_list(sl,l):
    sll=len(sl)
    for ind in (i for i,e in enumerate(l) if e==sl[0]):
        if l[ind:ind+sll]==sl:
            return ind,ind+sll
    return (-1,-1)

def capPassage(passage,answer,cap_length = 30):
    y = np.zeros(cap_length)
    left,right = find_sub_list(answer,passage)
    if(left==-1):
        return passage[0:cap_length]
    left = left - int((cap_length - len(answer))/2)
    right = right + int((cap_length + len(answer))/2)
    if(left < 0):
        left = 0
    if(right > len(passage)):
        right = len(passage)
    return passage[left:right]
    
def findAnsVec(answer,passage):
    ans = np.zeros((len(passage)))
    start,end = find_sub_list(answer,passage)
    if(start==-1):
        start = passage.index(answer[0])
        end = start + len(answer)
    ans[start:end] = 1
    return ans

In [6]:
X_train_comp_all = []
X_train_comp_ans_all = []
X_train_ans_all = []
Y_train_ques_all = []
invalid = 0
X_train_ans_label_all = []
for i,context in enumerate(contexts):
    passage = word_tokenize(context.lower())
    
    a_lab = np.zeros(len(passage))
    for j,_ in enumerate(qas[i]):
        answer = word_tokenize(qas[i][j]["answers"][0]['text'].lower())
        start,end = find_sub_list(answer,passage)
        if start == -1:
            continue
        a_lab[start:end+1] = 1
            
            
    for j,_ in enumerate(qas[i]):
        try:
            question = word_tokenize(qas[i][j]['question'].lower())
            answer = word_tokenize(qas[i][j]["answers"][0]['text'].lower())
            
            if CapPassage:
                cappedPassage = capPassage(passage,answer)
            else:
                cappedPassage = passage
            
            X_train_comp_ans_all.append(findAnsVec(answer,passage))
            X_train_ans_label_all.append(a_lab)
            X_train_comp_all.append(cappedPassage)
            X_train_ans_all.append(answer)
            Y_train_ques_all.append(question)
        except Exception as e:
            invalid = invalid+1
    

In [7]:
from collections import Counter
import operator
def findKMostFrequentWords(k):
    ctr = Counter([item for sublist in X_train_comp_all for item in sublist] + [item for sublist in Y_train_ques_all for item in sublist])
    sorted_ctr = sorted(ctr.items(), key=operator.itemgetter(1), reverse=True)
    return [item[0] for item in sorted_ctr[0:k]]

In [8]:
wordToTake = 50000
words = findKMostFrequentWords(100000)

In [9]:
_word_to_idx_reduced = {}
_idx_to_word_reduced = []


def _add_word_reduced(word):
    idx = len(_idx_to_word_reduced)
    _word_to_idx_reduced[word] = idx
    _idx_to_word_reduced.append(word)
    return idx


UNKNOWN_TOKEN = _add_word_reduced(UNKNOWN_WORD)
START_TOKEN = _add_word_reduced(START_WORD)
END_TOKEN = _add_word_reduced(END_WORD)




dimensions = glove.shape[1]
reduced_glove = []
reduced_glove.append(np.zeros(dimensions))
reduced_glove.append(-np.ones(dimensions))
reduced_glove.append(np.ones(dimensions))

for word in words:
    l = look_up_word(word)
    if(l != UNKNOWN_TOKEN):
        idx = _add_word_reduced(word)
        reduced_glove.append(glove[l])
        if(len(reduced_glove) == wordToTake):
            break
        
def look_up_word_reduced(word):
    return _word_to_idx_reduced.get(word, UNKNOWN_TOKEN)


def look_up_token_reduced(token):
    return _idx_to_word_reduced[token]

reduced_glove = np.array(reduced_glove)
reduced_glove.shape

(50000, 300)

In [10]:
!hostname
print(invalid)
for i in np.where(X_train_ans_label_all[110] == 1)[0]:
    print(X_train_comp_all[110][i])

cuda2.cims.nyu.edu
1510
rev
.
john
j.
cavanaugh
,
c.s.c
.
served
more
than
half
,
lobund
institute
for
animal
studies
and
medieval
institute
.
hall
of
liberal
arts
(


In [11]:
print(X_train_comp_all[0])
print(X_train_ans_all[0])

['architecturally', ',', 'the', 'school', 'has', 'a', 'catholic', 'character', '.', 'atop', 'the', 'main', 'building', "'s", 'gold', 'dome', 'is', 'a', 'golden', 'statue', 'of', 'the', 'virgin', 'mary', '.', 'immediately', 'in', 'front', 'of', 'the', 'main', 'building', 'and', 'facing', 'it', ',', 'is', 'a', 'copper', 'statue', 'of', 'christ', 'with', 'arms', 'upraised', 'with', 'the', 'legend', '``', 'ad', 'me', 'omnes', "''", '.', 'next', 'to', 'the', 'main', 'building', 'is', 'the', 'basilica', 'of', 'the', 'sacred', 'heart', '.', 'immediately', 'behind', 'the', 'basilica', 'is', 'the', 'grotto', ',', 'a', 'marian', 'place', 'of', 'prayer', 'and', 'reflection', '.', 'it', 'is', 'a', 'replica', 'of', 'the', 'grotto', 'at', 'lourdes', ',', 'france', 'where', 'the', 'virgin', 'mary', 'reputedly', 'appeared', 'to', 'saint', 'bernadette', 'soubirous', 'in', '1858.', 'at', 'the', 'end', 'of', 'the', 'main', 'drive', '(', 'and', 'in', 'a', 'direct', 'line', 'that', 'connects', 'through', '

In [12]:
find_sub_list(X_train_ans_all[0] , X_train_comp_all[0])

(101, 104)

In [13]:
print(invalid)
print(X_train_comp_all[101])
print(X_train_ans_all[101])
print(Y_train_ques_all[101])

c = list(zip(X_train_comp_all,X_train_comp_ans_all, X_train_ans_all, X_train_ans_label_all,Y_train_ques_all))
np.random.shuffle(c)
X_train_comp_all_shuffled,X_train_comp_ans_all_shuffled, X_train_ans_shuffled, X_train_ans_label_shuffled,Y_train_ques_all_shuffled = zip(*c)

print(X_train_comp_all_shuffled[101])
print(X_train_comp_ans_all_shuffled[101])
print(X_train_ans_shuffled[101])
print(X_train_ans_label_shuffled[101])
print(Y_train_ques_all_shuffled[101])


1510
['the', 'success', 'of', 'its', 'football', 'team', 'made', 'notre', 'dame', 'a', 'household', 'name', '.', 'the', 'success', 'of', 'note', 'dame', 'reflected', 'rising', 'status', 'of', 'irish', 'americans', 'and', 'catholics', 'in', 'the', '1920s', '.', 'catholics', 'rallied', 'up', 'around', 'the', 'team', 'and', 'listen', 'to', 'the', 'games', 'on', 'the', 'radio', ',', 'especially', 'when', 'it', 'knocked', 'off', 'the', 'schools', 'that', 'symbolized', 'the', 'protestant', 'establishment', 'in', 'america', '—', 'harvard', ',', 'yale', ',', 'princeton', ',', 'and', 'army', '.', 'yet', 'this', 'role', 'as', 'high-profile', 'flagship', 'institution', 'of', 'catholicism', 'made', 'it', 'an', 'easy', 'target', 'of', 'anti-catholicism', '.', 'the', 'most', 'remarkable', 'episode', 'of', 'violence', 'was', 'the', 'clash', 'between', 'notre', 'dame', 'students', 'and', 'the', 'ku', 'klux', 'klan', 'in', '1924.', 'nativism', 'and', 'anti-catholicism', ',', 'especially', 'when', 'dire

In [14]:
examples_to_take_train = 10000

X_train_comp = X_train_comp_all_shuffled[0:examples_to_take_train]
X_train_comp_ans = X_train_comp_ans_all_shuffled[0:examples_to_take_train]
X_train_ans = X_train_ans_shuffled[0:examples_to_take_train]
X_train_ans_label = X_train_ans_label_shuffled[0:examples_to_take_train]
Y_train_ques = Y_train_ques_all_shuffled[0:examples_to_take_train]
answer_indices = [np.where(x==1)[0].tolist() for x in X_train_comp_ans]

In [15]:
max_document_len = len(max(X_train_comp,key=len))
max_answer_len = len(max(X_train_ans,key=len))
max_question_len = len(max(Y_train_ques,key=len)) + 1

In [53]:
document_tokens = np.zeros((examples_to_take_train, max_document_len), dtype=np.int32)
document_lengths = np.zeros(examples_to_take_train, dtype=np.int32)
answer_labels = np.zeros((examples_to_take_train, max_document_len), dtype=np.int32)
answer_masks = np.zeros((examples_to_take_train, max_answer_len, max_document_len), dtype=np.int32)
answer_lengths = np.zeros(examples_to_take_train, dtype=np.int32)
question_input_tokens = np.zeros((examples_to_take_train, max_question_len), dtype=np.int32)
question_output_tokens = np.zeros((examples_to_take_train, max_question_len), dtype=np.int32)
question_lengths = np.zeros(examples_to_take_train, dtype=np.int32)
suppression_answer = np.zeros((examples_to_take_train, reduced_glove.shape[0], 1),dtype=np.int32)
expression_contexts = np.ones((examples_to_take_train, reduced_glove.shape[0], 1),dtype=np.int32)

In [54]:
print(answer_labels.shape)
for i in range(examples_to_take_train):
    answer_labels[i,0:len(X_train_ans_label[i])] = X_train_ans_label[i]
    for j, word in enumerate(X_train_comp[i]):
        document_tokens[i, j] = look_up_word_reduced(word)
    document_lengths[i] = len(X_train_comp[i])

    for j, index in enumerate(answer_indices[i]):
        answer_masks[i, j, index] = 1
    answer_lengths[i] = len(answer_indices[i])
    
    #print(Y_train_ques[i])
    question_input_words = ([START_WORD] + Y_train_ques[i])
    question_output_words = (Y_train_ques[i] + [END_WORD])

    for j, word in enumerate(question_input_words):
            question_input_tokens[i, j] = look_up_word_reduced(word)
    for j, word in enumerate(question_output_words):
        question_output_tokens[i, j] = look_up_word_reduced(word)
    question_lengths[i] = len(question_input_words)
    
    for j, word in enumerate(X_train_ans[i]):
        if(word not in Y_train_ques[i]):
            suppression_answer[i, look_up_word_reduced(word),:] = 1

    for j,word in enumerate(X_train_comp[i]):
        expression_contexts[i,look_up_word_reduced(word),:] = 0

(1000, 380)


In [55]:
def create_vocabulary(data):
    flat_list = [item for sublist in data for item in sublist]
    vocabulary = sorted(set(flat_list))
    vocabulary.append("<UNK>")
    vocabulary.append("unk")
    vocabulary.append("eos")
    vocabulary = ["<EOS>"] + vocabulary
    word_to_index = { word:i for i,word in enumerate(vocabulary) }
    index_to_word = { i:word for i,word in enumerate(vocabulary) }
    return (vocabulary,word_to_index,index_to_word)

In [56]:
print(reduced_glove.shape)
vocabulary_comp,word_to_index_comp,index_to_word_comp = create_vocabulary(X_train_comp + Y_train_ques)
print(len(vocabulary_comp))
print(word_to_index_comp["?"])
print(word_to_index_comp["what"])

(50000, 300)
18501
1435
17986


In [57]:
def create_one_hot_vector(data,vocabulary,word_to_index,index_to_word, maxLen):
    one_hot = np.zeros([maxLen,len(vocabulary)])
    for i,word in enumerate(data):
        if i >= maxLen:
            break
        if(word not in word_to_index):
            word = "<UNK>"
        one_hot[i][word_to_index[word]] = 1
    return one_hot

def create_one_hot_vector_from_indices(data,maxLen,vocabulary):
    one_hot = np.zeros([maxLen,len(vocabulary)])
    for i,indice in enumerate(data):
        if i >= maxLen:
            break
        one_hot[i][int(indice)] = 1
    return one_hot


def create_one_hot_training_Set(data,maxLen,vocabulary):
    one_hot_data = np.zeros([data.shape[0],maxLen,len(vocabulary)])
    for i in range(data.shape[0]):
        one_hot_data[i] = create_one_hot_vector_from_indices(data[i],maxLen,vocabulary)
    return one_hot_data




In [58]:
def sentences_to_indices_glove(X,max_len):
    
    m = len(X)                                 
    
    X_indices = np.full([m,max_len],look_up_word_reduced(END_WORD))
    
    for i in range(m):
        j = 0
        for w in X[i]:
            if(j>=max_len):
                break;
            
            X_indices[i, j] = look_up_word_reduced(w)
            j = j+1
    return X_indices

In [59]:
document_tokens = sentences_to_indices_glove(X_train_comp, max_document_len)

In [60]:
document_tokens[0]

array([  49, 1219,   11, 2541,  109,   10, 2979,   95,    4,   90,   13,
          3,   49, 1219, 2979,  353,   15,    0,   14,    6,   46,  732,
        217,    5,    3, 7674,    5, 1219,   27, 3550,   18,    3, 2979,
       1752,    5, 1219,   15,    0,   14,    6,  102,    4,    3,  938,
       7674,    5, 1219,   11,  682,   90,   13,   49, 1219,    8, 1351,
          9,  304, 1219,    6,    2,    2,    2,    2,    2,    2,    2,
          2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,
          2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,
          2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,
          2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,
          2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,
          2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,
          2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,
          2,    2,    2,    2,    2,    2,    2,   

In [61]:
answer_labels.shape

(1000, 380)

## Model

In [107]:
cellFlag = 'GRU'
QuestionLossFlag = True
SupressionLossFlag = False
ExpressionLossFlag = False
minQuestionLoss = 1.0 

### Document Embedding

In [108]:
import tensorflow.contrib.seq2seq as seq2seq

tf.reset_default_graph()

embedding = tf.get_variable("embedding", initializer=reduced_glove)
embedding = tf.cast(embedding, dtype=tf.float32)
EMBEDDING_DIMENS = glove.shape[1]

d_tokens = tf.placeholder(tf.int32, shape=[None, None], name="d_tokens")
d_lengths = tf.placeholder(tf.int32, shape=[None], name="d_lengths")


document_emb = tf.nn.embedding_lookup(embedding, d_tokens, name="document_emb")
document_emb = tf.cast(document_emb, dtype=tf.float64, name="casted_document_emb")


### Answer Generation

In [109]:
if cellFlag == 'LSTM':
    forward_cell = tf.contrib.rnn.LSTMCell(EMBEDDING_DIMENS)
    backward_cell = tf.contrib.rnn.LSTMCell(EMBEDDING_DIMENS)
elif cellFlag == 'GRU':
    forward_cell = tf.contrib.rnn.GRUCell(EMBEDDING_DIMENS)
    backward_cell = tf.contrib.rnn.GRUCell(EMBEDDING_DIMENS)

answer_outputs, states = tf.nn.bidirectional_dynamic_rnn(
    forward_cell, backward_cell, document_emb, d_lengths, dtype=tf.float64,
    scope="answer_rnn")

answer_outputs = tf.concat(answer_outputs, 2, name="answer_output_concat")

answer_outputs = tf.cast(answer_outputs,tf.float32, name="answer_output_concat")

answer_tags = tf.layers.dense(inputs=answer_outputs, units=2, name="answer_tags")


a_labels = tf.placeholder(tf.int32, shape=[None, None], name="a_labels")

answer_mask = tf.sequence_mask(d_lengths, dtype=tf.float32, name="answer_mask")

answer_loss = seq2seq.sequence_loss(
    logits=answer_tags, targets=a_labels, weights=answer_mask, name="answer_loss")

answer_loss = tf.Print(answer_loss, [answer_loss], message="This is answer_loss: ")

## Question Generation

### Encoder

In [110]:
encoder_input_mask = tf.placeholder(
    tf.float32, shape=[None, None, None], name="encoder_input_mask")
encoder_inputs = tf.matmul(encoder_input_mask, answer_outputs, name="encoder_inputs")
encoder_lengths = tf.placeholder(tf.int32, shape=[None], name="encoder_lengths")

if cellFlag == 'GRU':
    encoder_cell = tf.contrib.rnn.GRUCell(forward_cell.state_size + backward_cell.state_size)
elif cellFlag == 'LSTM':
    encoder_cell = tf.contrib.rnn.LSTMCell(2 * EMBEDDING_DIMENS)



In [111]:
_, encoder_state = tf.nn.dynamic_rnn(
    encoder_cell, encoder_inputs, encoder_lengths, dtype=tf.float32, scope="encoder_rnn")

### Decoder

In [112]:
from tensorflow.python.layers.core import Dense


decoder_inputs = tf.placeholder(tf.int32, shape=[None, None], name="decoder_inputs")
decoder_lengths = tf.placeholder(tf.int32, shape=[None], name="decoder_lengths")

decoder_emb = tf.nn.embedding_lookup(embedding, decoder_inputs,name="decoder_embedding")
decoder_emb = tf.cast(decoder_emb,tf.float32,name="decoder_embedding_cast")

helper = seq2seq.TrainingHelper(decoder_emb , decoder_lengths, name="helper")


projection = Dense(embedding.shape[0], use_bias=False, name="projection")

if cellFlag == 'GRU':
    decoder_cell = tf.contrib.rnn.GRUCell(encoder_cell.state_size)
elif cellFlag == "LSTM":
    decoder_cell = tf.contrib.rnn.LSTMCell(2 * EMBEDDING_DIMENS)

In [113]:
decoder = seq2seq.BasicDecoder(decoder_cell, helper, encoder_state, output_layer=projection)
decoder_outputs, _, _ = seq2seq.dynamic_decode(decoder, scope="decoder")
decoder_outputs = decoder_outputs.rnn_output

decoder_labels = tf.placeholder(tf.int64, shape=[None, None], name="decoder_labels")
question_mask = tf.sequence_mask(decoder_lengths ,dtype=tf.float32)

## Question Generation Loss

In [115]:
# NLL Loss
question_loss = seq2seq.sequence_loss(
    logits=decoder_outputs, targets=decoder_labels, weights=question_mask,
    name="question_loss")
question_loss = tf.Print(question_loss, [question_loss], message="This is question_loss: ")

#Suppression Loss
s_answer = tf.placeholder(tf.float32, shape=[None,None,None], name="suppression_answer")
lambdaSuppress = 0.5

suppression_loss = lambdaSuppress * tf.reduce_sum(tf.matmul(tf.exp(decoder_outputs), s_answer))
suppression_loss = tf.Print(suppression_loss, [suppression_loss], message="This is suppression_loss: ")


#Expression Loss
e_context = tf.placeholder(tf.float32, shape=[None,None,None], name="expression_answer")
lambdaExpress = 0.1

expression_loss = lambdaExpress * tf.reduce_sum(tf.matmul(tf.exp(decoder_outputs), e_context))
expression_loss = tf.Print(expression_loss, [expression_loss], message="This is expression loss: ")

In [116]:
loss = answer_loss
if QuestionLossFlag:
    loss = loss + question_loss
if SupressionLossFlag:
    loss = loss + suppression_loss
if ExpressionLossFlag:
    loss = loss + expression_loss

In [101]:
#document_tokens = np.zeros((examples_to_take_train, max_document_len), dtype=np.int32)
#document_lengths = np.zeros(examples_to_take_train, dtype=np.int32)
#answer_labels = np.zeros((examples_to_take_train, max_document_len), dtype=np.int32)
#answer_masks = np.zeros((examples_to_take_train, max_answer_len, max_document_len), dtype=np.int32)
#answer_lengths = np.zeros(examples_to_take_train, dtype=np.int32)
#question_input_tokens = np.zeros((examples_to_take_train, max_question_len), dtype=np.int32)
#question_output_tokens = np.zeros((examples_to_take_train, max_question_len), dtype=np.int32)
#question_lengths = np.zeros(examples_to_take_train, dtype=np.int32)
#suppression_answer = np.zeros((examples_to_take_train,max_answer_len),dtype=np.int32)

In [102]:
def shuffle_list(*ls):
    l =list(zip(*ls))
    np.random.shuffle(l)
    return zip(*l)

In [117]:
import math
def createBatch(inputs,batch_size,shuffle=False):
    outputs = []
    num_batches = math.ceil(len(inputs[0])/batch_size)
    
    for index,inp in enumerate(inputs):
        start = 0
        output = []
        for i in range(num_batches-1):
            maxD = max(inputs[1][start:start+batch_size])
            maxA = max(inputs[4][start:start+batch_size])
            maxQ = max(inputs[7][start:start+batch_size])
            if index == 0 or index == 2:
                output.append(inp[start:start+batch_size,0:maxD]) 
            elif index==3:
                output.append(inp[start:start+batch_size,0:maxA,0:maxD]) 
            elif index==5 or index==6:
                output.append(inp[start:start+batch_size,0:maxQ])
            elif index == 8: # 
                output.append(inp[start:start+batch_size,:,:]) 
            else: 
                output.append(inp[start:start+batch_size])
            start = start + batch_size
        
        # Remaining training sample i.e. training mod batch_size
        maxD = max(inputs[1][start:])
        maxA = max(inputs[4][start:])
        maxQ = max(inputs[7][start:])
        if index == 0 or index == 2:
            output.append(inp[start:,0:maxD]) 
        elif index==3:
            output.append(inp[start:,0:maxA,0:maxD]) 
        elif index==5 or index==6:
            output.append(inp[start:,0:maxQ]) 
        elif index ==8:
            output.append(inp[start:,:, :]) 
        else: 
            output.append(inp[start:])
        outputs.append(output)
    
    return outputs


In [118]:
batch_size = 32
batch_input = createBatch([document_tokens,document_lengths,answer_labels,answer_masks,answer_lengths,question_input_tokens,question_output_tokens,question_lengths,suppression_answer,expression_contexts]
                    ,batch_size)

In [119]:
print("No of features:",len( batch_input))
print("No of batches:",len( batch_input[0]))

No of features: 10
No of batches: 32


In [ ]:
optimizer = tf.train.AdamOptimizer(learning_rate=3e-3).minimize(loss)

saver = tf.train.Saver()
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.InteractiveSession(config=config)

session.run(tf.global_variables_initializer())
# session.run(tf.variables_initializer(saved_vars))


EPOCHS = 20

for epoch in range(1, EPOCHS + 1):
    print("Epoch {0}".format(epoch))
    for batchNum in range(len(batch_input[0])):
        print("Batch : ",batchNum)
        t = session.run([optimizer, loss, question_loss, suppression_loss, expression_loss], {
            d_tokens: batch_input[0][batchNum],
            d_lengths: batch_input[1][batchNum],
            a_labels: batch_input[2][batchNum],
            encoder_input_mask: batch_input[3][batchNum],
            encoder_lengths: batch_input[4][batchNum],
            decoder_inputs: batch_input[5][batchNum],
            decoder_labels: batch_input[6][batchNum],
            decoder_lengths: batch_input[7][batchNum],
            s_answer: batch_input[8][batchNum],
            e_context: batch_input[9][batchNum],
        })
        print("Loss: {0}".format(t[1]))
    if t[2] < minQuestionLoss:
        SuppresionLossFlag = True
        ExpressionLossFlag = True
    if(epoch%5 == 0):
        print("Saving model")
        #saver.save(session, "qgen-model")


Epoch 1
Batch :  0
Loss: 11.520698547363281
Batch :  1
Loss: 11.572921752929688
Batch :  2
Loss: 9.996600151062012
Batch :  3
Loss: 8.306029319763184
Batch :  4
Loss: 7.818546295166016
Batch :  5
Loss: 8.085103034973145
Batch :  6
Loss: 8.181066513061523
Batch :  7
Loss: 8.76831340789795
Batch :  8
Loss: 8.270374298095703
Batch :  9
Loss: 8.027091026306152
Batch :  10
Loss: 8.10173511505127
Batch :  11
Loss: 7.902759552001953
Batch :  12
Loss: 7.485025882720947
Batch :  13
Loss: 7.334495544433594
Batch :  14
Loss: 7.30300760269165
Batch :  15
Loss: 7.475589752197266
Batch :  16
Loss: 7.3701701164245605
Batch :  17
Loss: 7.261754512786865
Batch :  18
Loss: 7.24669075012207
Batch :  19
Loss: 7.213845729827881
Batch :  20
Loss: 7.165980815887451
Batch :  21
Loss: 7.403918266296387
Batch :  22
Loss: 7.080841064453125
Batch :  23
Loss: 7.249973297119141
Batch :  24
Loss: 7.281829357147217
Batch :  25
Loss: 7.606300354003906
Batch :  26
Loss: 7.197308540344238
Batch :  27
Loss: 7.43951654434

Loss: 3.72831392288208
Batch :  4
Loss: 3.69512939453125
Batch :  5
Loss: 3.934354066848755
Batch :  6
Loss: 3.7205159664154053
Batch :  7
Loss: 4.058916091918945
Batch :  8
Loss: 3.8494694232940674
Batch :  9
Loss: 3.8997690677642822
Batch :  10
Loss: 3.9186880588531494
Batch :  11
Loss: 3.905054807662964
Batch :  12
Loss: 3.7119715213775635
Batch :  13
Loss: 3.7353968620300293
Batch :  14
Loss: 3.5887691974639893
Batch :  15
Loss: 3.814040422439575
Batch :  16
Loss: 3.544013738632202
Batch :  17
Loss: 3.540208578109741
Batch :  18
Loss: 3.3428831100463867
Batch :  19
Loss: 3.3640565872192383
Batch :  20
Loss: 3.335348129272461
Batch :  21
Loss: 3.4488658905029297
Batch :  22
Loss: 3.3032944202423096
Batch :  23
Loss: 3.4964327812194824
Batch :  24
Loss: 3.6728591918945312
Batch :  25
Loss: 3.728938102722168
Batch :  26
Loss: 3.4259185791015625
Batch :  27
Loss: 3.3911325931549072
Batch :  28
Loss: 3.5445642471313477
Batch :  29
Loss: 3.4456288814544678
Batch :  30
Loss: 3.61517739295

In [ ]:
saver = tf.train.Saver()
saver.save(session, "qgen-model")

In [ ]:
batch_input[3][0].shape

In [ ]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)
saver = tf.train.Saver()

saver.restore(session, 'qgen-model')

In [43]:
answers = session.run(answer_tags, {
    d_tokens: batch_input[0][2],
    d_lengths: batch_input[1][2],
})
answers = np.argmax(answers, 2)

In [48]:
for i in range(32):
    print("Prediction")
    printAllAns(answers,2,0)
    print("Ground Truth")
    printAllAns(batch_input[2][2],2,0)
answers[0]

Prediction
 
Ground Truth
idol gives back is season six featuring $ 185 million in  
Prediction
 
Ground Truth
idol gives back is season six featuring $ 185 million in  
Prediction
 
Ground Truth
idol gives back is season six featuring $ 185 million in  
Prediction
 
Ground Truth
idol gives back is season six featuring $ 185 million in  
Prediction
 
Ground Truth
idol gives back is season six featuring $ 185 million in  
Prediction
 
Ground Truth
idol gives back is season six featuring $ 185 million in  
Prediction
 
Ground Truth
idol gives back is season six featuring $ 185 million in  
Prediction
 
Ground Truth
idol gives back is season six featuring $ 185 million in  
Prediction
 
Ground Truth
idol gives back is season six featuring $ 185 million in  
Prediction
 
Ground Truth
idol gives back is season six featuring $ 185 million in  
Prediction
 
Ground Truth
idol gives back is season six featuring $ 185 million in  
Prediction
 
Ground Truth
idol gives back is season six featuring

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0,

In [45]:
def printDoc(batch,num):
    for i in batch_input[0][batch][num]:
        print(look_up_token_reduced(i),sep=" ", end=" ")
    print(" ")

def printQues(batch,num):
    for i in batch_input[5][batch][num]:
        print(look_up_token_reduced(i),sep=" ", end=" ")
    print(" ")
    
def printAnsForQuestion(batch, num):
    for i in batch_input[5][batch][num]:
        print(look_up_token_reduced(i),sep=" ", end=" ")
    print(" ")
    
def printAllAns(answers, batch, num):
    for i,word in enumerate(batch_input[0][batch][num]):
        if answers[num][i] == 1 :
            print(look_up_token_reduced(word),sep=" ", end=" ")
    print(" ")
    

In [87]:
import itertools

batchNum = 2

helper = seq2seq.GreedyEmbeddingHelper(embedding, tf.fill([batch_input[0][batchNum].shape[0]], START_TOKEN), END_TOKEN)
decoder = seq2seq.BasicDecoder(decoder_cell, helper, encoder_state, output_layer=projection)
decoder_outputs, _, _ = seq2seq.dynamic_decode(decoder, maximum_iterations=max_question_len)
decoder_outputs = decoder_outputs.rnn_output


questions = session.run(decoder_outputs, {
    d_tokens: batch_input[0][batchNum],
    d_lengths: batch_input[1][batchNum],
    a_labels: batch_input[2][batchNum],
    encoder_input_mask: batch_input[3][batchNum],
    encoder_lengths: batch_input[4][batchNum],
})



In [88]:
batch_input[0][batchNum].shape

(32, 333)

In [89]:
#questions[:,:,END_TOKEN] = 0
qs = np.argmax(questions, 2)

In [90]:
# q = set()
l = []
for i in range(batch_input[0][batchNum].shape[0]):
    print("---------------------------------------------------------------------------------------------")
    question = itertools.takewhile(lambda t: t != END_TOKEN, qs[i])
    print("Generated Question: " + " ".join(look_up_token_reduced(token) for token in question))
    print("Ground Truth Question: ")
    printQues(batchNum,i)
    print("Ground Truth Answer: ", X_train_ans_shuffled[batch_size*batchNum + i])
    print("Context:")
    printDoc(batchNum,i)
    print("---------------------------------------------------------------------------------------------")
    
    
# q

---------------------------------------------------------------------------------------------
Generated Question: what the the the the the the the the the the the the the the the the the the the the the the the the the the the the
Ground Truth Question: 
<START> how much money has american idol raised for charity with its idol gives back specials ? <UNK> <UNK> <UNK> <UNK>  
Ground Truth Answer:  ['$', '185', 'million']
Context:
idol gives back is a special charity event started in season six featuring performances by celebrities and various fund-raising initiatives . this event was also held in seasons seven and nine and has raised nearly $ 185 million in total . <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END

the city management and urban policy program was ranked first in the nation , and the special education program second , by u.s. news & world report 's 2016 rankings . usn & <UNK> also ranked several programs in the top 25 among u.s. universities . <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <

greece was eventually occupied by the nazis who proceeded to administer athens and thessaloniki , while other regions of the country were given to nazi germany 's partners , fascist italy and bulgaria . the occupation brought about terrible hardships for the greek civilian population . over 100,000 civilians died of starvation during the winter of <UNK> , tens of thousands more died because of reprisals by nazis and collaborators , the country 's economy was ruined and the great majority of greek jews were deported and murdered in nazi concentration camps . the greek resistance , one of the most effective resistance movements in europe fought vehemently against the nazis and their collaborators . the german occupiers committed numerous atrocities , mass executions , and wholesale slaughter of civilians and destruction of towns and villages in reprisals . in the course of the concerted <UNK> campaign , hundreds of villages were systematically torched and almost 1,000,000 greeks left hom

in march 1861 , victoria 's mother died , with victoria at her side . through reading her mother 's papers , victoria discovered that her mother had loved her deeply ; she was heart-broken , and blamed conroy and lehzen for `` wickedly '' estranging her from her mother . to relieve his wife during her intense and deep grief , albert took on most of her duties , despite being ill himself with chronic stomach trouble . in august , victoria and albert visited their son , the prince of wales , who was attending army manoeuvres near dublin , and spent a few days holidaying in killarney . in november , albert was made aware of gossip that his son had slept with an actress in ireland . appalled , albert travelled to cambridge , where his son was studying , to confront him . by the beginning of december , albert was very unwell . he was diagnosed with typhoid fever by william jenner , and died on 14 december <UNK> victoria was devastated . she blamed her husband 's death on worry over the prin

the greek-speaking orthodox have collected canons and commentaries upon them in a work known as the <UNK> ( greek : <UNK> , `` rudder '' ) , so named because it is meant to `` steer '' the church . the orthodox christian tradition in general treats its canons more as guidelines than as laws , the bishops adjusting them to cultural and other local circumstances . some orthodox canon scholars point out that , had the ecumenical councils ( which <UNK> in greek ) meant for the canons to be used as laws , they would have called them <UNK> ( laws ) rather than <UNK> ( rules ) , but almost all orthodox conform to them . the dogmatic decisions of the councils , though , are to be obeyed rather than to be treated as guidelines , since they are essential for the church 's unity . <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <END> <EN

---------------------------------------------------------------------------------------------
Generated Question: what the the the the the the the the the the the the the the the the the the the the the the the the the the the the
Ground Truth Question: 
<START> how many legat offices does the fbi operate ? <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK>  
Ground Truth Answer:  ['60']
Context:
despite its domestic focus , the fbi also maintains a significant international footprint , operating 60 legal <UNK> ( legat ) offices and 15 <UNK> in u.s. embassies and consulates across the globe . these overseas offices exist primarily for the purpose of coordination with foreign security services and do not usually conduct unilateral operations in the host countries . the fbi can and does at times carry out secret activities overseas , just as the cia has a limited domestic function ; these activities generally require coordination across government agencies . <END> <END> <E

severe weather occurs regularly in north carolina . on the average , a hurricane hits the state once a decade . destructive hurricanes that have struck the state include hurricane fran , hurricane floyd , and hurricane hazel , the strongest storm to make landfall in the state , as a category 4 in <UNK> hurricane isabel stands out as the most damaging of the 21st century . tropical storms arrive every 3 or 4 years . in addition , many hurricanes and tropical storms graze the state . in some years , several hurricanes or tropical storms can directly strike the state or brush across the coastal areas . only florida and louisiana are hit by hurricanes more often . although many people believe that hurricanes menace only coastal areas , the rare hurricane which moves inland quickly enough can cause severe damage ; for example , in 1989 , hurricane hugo caused heavy damage in charlotte and even as far inland as the blue ridge mountains in the northwestern part of the state . on the average ,

emotions involve different components , such as subjective experience , cognitive processes , expressive behavior , psychophysiological changes , and instrumental behavior . at one time , academics attempted to identify the emotion with one of the components : william james with a subjective experience , behaviorists with instrumental behavior , <UNK> with physiological changes , and so on . more recently , emotion is said to consist of all the components . the different components of emotion are categorized somewhat differently depending on the academic discipline . in psychology and philosophy , emotion typically includes a subjective , conscious experience characterized primarily by psychophysiological expressions , biological reactions , and mental states . a similar <UNK> description of emotion is found in sociology . for example , peggy <UNK> described emotions as involving physiological components , cultural or emotional labels ( e.g. , anger , surprise etc . ) , expressive body

In [ ]:
batch_input[5][18].shape

In [ ]:
batch_input[5][1].shape

In [ ]:
batch_input[6][1].shape

In [ ]:
max(batch_input[7][1])

In [ ]:
max_question_len

In [ ]:
len(batch_input)

In [ ]:
reduced_glove